In [1]:
import os
import copy
import json
import numpy as np
import pandas as pd
from datetime import datetime
from tqdm import tqdm
from sys import platform

import warnings
warnings.filterwarnings('ignore')

# unicode minus를 사용하지 않기 위한 설정 (minus 깨짐현상 방지)
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams["figure.autolayout"] = True

# 설치된 폰트 출력
import matplotlib.font_manager as fm
font_list = [font.name for font in fm.fontManager.ttflist]
plt.rcParams['font.family'] = 'Malgun Gothic'


BASE_DIR = os.getcwd()
print('>> Current OS: ', platform)
print('>> Current WD: ', BASE_DIR)

/Users/huni/.pyenv/versions/3.8.2/envs/{huni}/lib/python3.8/site-packages/pandas/compat/__init__.py:97: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


>> Current OS:  darwin
>> Current WD:  /Users/huni/Proj/Research/MixedRF


In [9]:
class Load:
    def __init__(self, codeBook):
        if 'darwin' in platform:
            self.BASE_DIR = '/Users/huni/Dropbox/[3]Project/[혼합효과 랜덤포레스트_2022]'
        else:
            self.BASE_DIR = r'C:\Users\jhun1\Dropbox\[3]Project\[혼합효과 랜덤포레스트_2022]'
        
        global BASE_DIR
        codebook_Folder = 'drive-download-20220816T053902Z-001'

        print('>>>>> Init: load raw data')
        self.rawStu = pd.read_excel(os.path.join(BASE_DIR, 'data', 'rawData(HK).xlsx'), sheet_name='stu')
        self.rawSCH = pd.read_excel(os.path.join(BASE_DIR, 'data', 'rawData(HK).xlsx'), sheet_name='sch')
        self.rawTCH = pd.read_excel(os.path.join(BASE_DIR, 'data', 'rawData(HK).xlsx'), sheet_name='tch')
        self.dataLS = [self.rawStu, self.rawSCH, self.rawTCH]

        # desciptive
        print('>> Stu data set', self.rawStu.shape)
        print('>> Sch data set', self.rawSCH.shape)
        print('>> Tch data set', self.rawTCH.shape)
        
        
        self.cb = pd.read_excel(os.path.join(self.BASE_DIR, codebook_Folder, codeBook),
                            skiprows=[0] # 맨 윗줄 제거
                            )


Loader = Load(codeBook='TargetPaper_CODEBOOK.xlsx')

>>>>> Init: load raw data
>> Stu data set (6037, 1119)
>> Sch data set (152, 197)
>> Tch data set (3754, 351)


In [16]:
class Preprocessing(Load):
    def __init__(self, dataLS, cb):
        # super().__init__(dataLS, codebook = cb)
        self.dataLS = copy.deepcopy(dataLS) # if you not copy, object 'Loader' is damaged
        self.cb = cb
        self.testBook = {
            'read/math/sci_1': 'PV1MATH PV1READ PV1SCIE'.split(),
            'read/math_1': 'PV1MATH PV1READ'.split(),
            'read/math_10': 'PV1MATH PV2MATH PV3MATH PV4MATH PV5MATH PV6MATH PV7MATH PV8MATH PV9MATH PV10MATH PV1READ PV2READ PV3READ PV4READ PV5READ PV6READ PV7READ PV8READ PV9READ PV10READ'.split(),
            'read_1': ['PV1READ'],
            'read_10': 'PV1READ PV2READ PV3READ PV4READ PV5READ PV6READ PV7READ PV8READ PV9READ PV10READ'.split()
        }
        self.threshold = {'academic': None, 'escs': None}

    def calculate_academic_score(self, testKey):
        ### calculate academic score and mapping whether resilient or not
        print('\n\n>>>> Calculate: academic score and its threshold')
        before = self.dataLS[0].shape[1]
        
        self.dataLS[0]['academic_score'] = self.dataLS[0].loc[:, self.testBook[testKey]].mean(axis=1)


        def threshold(col_academic): # escs 는 계산할 필요 없음
            thres_val = col_academic.quantile(0.75) # 반드시 escs 자르기 전에 계산해야 함
            return thres_val
        
        threshold_academic = threshold(self.dataLS[0]['academic_score'])
        print('> Threshold: academic score, ', threshold_academic)

        self.threshold['academic'] = threshold_academic
        after = self.dataLS[0].shape[1]
        assert after - before == 1
        return None


    def defaultCleaner(self):
        print('\n\n>>>> Cleaning: select variable')

        ### slicing only codebook independent var (with academic score)
        def drop_Unidentified_variable(codebook):
            variable_name = list(codebook['variable_name'].values)
            print('>>>> 1. left only identified variable', len(variable_name))
            print('> academic_score variable should be left')
            new = [variable for variable in variable_name if variable != '?']
            new.append('academic_score') # Imp
            
            print('> cleaned :', len(new))
            return new


        def cleaningVariable(dataLS, using_variable_list):
            print('\n>>>> 2. drop useless variable')

            """
            1. iteration through data set (stu, sch, tch)
            2. iteration through every column
            3. save column when it is contained
            """
            new_data_ls = {'Stu': [], 'Sch': [], 'Tch': []}

            count = 0
            for data_set, label in zip(dataLS, new_data_ls.keys()):
                toDrop = []
                for col in data_set.columns:
                    if col in using_variable_list:
                        count += 1
                    
                    elif col == 'ESCS':
                        continue

                    else:
                        toDrop.append(col)
                
                newDF = data_set.drop(toDrop, axis=1)
                new_data_ls[label] = newDF
                print(f'> {label} data only left.. : ', len(newDF.columns))


            assert count == len(using_variable_list), print('*error: ', count, "...", len(using_variable_list))
            
            return new_data_ls


        var_ls =  drop_Unidentified_variable(self.cb) # cleaening unidentified variable
        output = cleaningVariable(dataLS = self.dataLS, using_variable_list=var_ls)
        self.cleaned = output['Stu']
        assert type(self.cleaned) == pd.DataFrame

        return None


    def slice(self):
        print('\n\n>>>> Slice: slice by escs')
        self.threshold['escs'] = self.cleaned['ESCS'].quantile(0.25)
        df = copy.deepcopy(self.cleaned)
        before = df.shape[0]

        toDrop = []
        for idx, val in zip(df.index, df['ESCS'].values):
            if val < self.threshold['escs']:
                continue
            else:
                toDrop.append(idx)

        output = df.drop(toDrop, axis = 0)
        after = output.shape[0]
        print('>> before: ', before, '>> after: ', after)
        self.sliced = output
        return output
    
    def coding_resilient(self):
        df = copy.deepcopy(self.sliced)
        df.reset_index(drop=True, inplace=True) # in case of error, clear index
        
        for idx, val in enumerate(df['academic_score']):
            if val >= self.threshold['academic']:
                df.loc[idx, 'resilient'] = 1
            else:
                df.loc[idx, 'resilient'] = 0

        total = df.shape[0]
        count_resilient = df[df['resilient']==1].shape[0]
        count_not_resilient = df[df['resilient']==0].shape[0]

        def percentage(inputC, totalC):
            return np.round((inputC/totalC)*100, 1)
        print('> resilient students: ', count_resilient, f'({percentage(count_resilient, total)}%)')
        print('> not resilient students: ', count_not_resilient, f'({percentage(count_not_resilient, total)}%)')


        df.drop(['academic_score'], axis=1, inplace=True) # after calculating is done, drop score column
        return df

    
preprocessor = Preprocessing(dataLS = Loader.dataLS, cb = Loader.cb)
preprocessor.calculate_academic_score('read_10') # should be calculated before drop
preprocessor.defaultCleaner()
df_sliced = preprocessor.slice()
df = preprocessor.coding_resilient()



>>>> Calculate: academic score and its threshold
> Threshold: academic score,  594.5394


>>>> Cleaning: select variable
>>>> 1. left only identified variable 30
> academic_score variable should be left
> cleaned : 29

>>>> 2. drop useless variable
> Stu data only left.. :  30
> Sch data only left.. :  0
> Tch data only left.. :  0


>>>> Slice: slice by escs
>> before:  6037 >> after:  1459
> resilient students:  242 (16.6%)
> not resilient students:  1217 (83.4%)


In [17]:
def minor_adjust(inputDf, highlight_column):
    assert type(highlight_column) == str, print('> this code only work for adjusting only one column')
    
    output = copy.deepcopy(inputDf)
    output.set_index(highlight_column, drop=True, inplace=True)
    output.reset_index(drop=False, inplace=True)
    return output

cleaned_df = minor_adjust(df, 'resilient')

In [18]:
cleaned_df.to_excel(os.path.join(BASE_DIR, 'data', 'cleanedData(HK).xlsx'),
            index=False)

In [ ]:
# SPSS 26.0의 MCMC 사용해서 결측치 대체함 
# https://www.statisticshowto.com/missing-values-spss/


# reading achivement top 25%를 resilient로 했다는데 한문항만 썼는지, 다른 문항들 전체를 썼는지는 확인 필요함